## CNN介绍
Convolutional Neural Networks翻译为卷积神经网络，常用在图像识别和语音分析等领域

In [1]:
import tensorflow as tf
import numpy as np

## MNIST数据集

### 加载MNIST数据集

In [3]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('E:\\src\\pycharm\\tensorflow-learn\\data\\mnist', one_hot=True)

Extracting E:\src\pycharm\tensorflow-learn\data\mnist\train-images-idx3-ubyte.gz
Extracting E:\src\pycharm\tensorflow-learn\data\mnist\train-labels-idx1-ubyte.gz
Extracting E:\src\pycharm\tensorflow-learn\data\mnist\t10k-images-idx3-ubyte.gz
Extracting E:\src\pycharm\tensorflow-learn\data\mnist\t10k-labels-idx1-ubyte.gz


In [4]:
n_output_layer = 10  #输出类别为10

### 定义待训练的卷积神经网络

In [5]:
def convolutional_neural_network(data):
    weights = {'w_conv1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
              'w_conv2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
              'w_fc': tf.Variable(tf.random_normal([7*7*64, 1024])),
              'out': tf.Variable(tf.random_normal([1024, n_output_layer]))}
    biases = {'b_conv1': tf.Variable(tf.random_normal([32])),
             'b_conv2': tf.Variable(tf.random_normal([64])),
             'b_fc': tf.Variable(tf.random_normal([1024])),
             'out': tf.Variable(tf.random_normal([n_output_layer]))}
    
    data = tf.reshape(data, [-1, 28, 28, 1])
    
    conv1 = tf.nn.relu(tf.add(tf.nn.conv2d(data, weights['w_conv1'], strides=[1,1,1,1], padding='SAME'), biases['b_conv1']))
    conv1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    conv2 = tf.nn.relu(tf.add(tf.nn.conv2d(conv1, weights['w_conv2'], strides=[1,1,1,1], padding='SAME'), biases['b_conv2']))
    conv2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    fc = tf.reshape(conv2, [-1, 7*7*64])
    fc = tf.nn.relu(tf.add(tf.matmul(fc, weights['w_fc']), biases['b_fc']))
    
    # dropout剔除一些“神经元”
    #fc = tf.nn.dropout(fc, 0.8)
    
    output = tf.add(tf.matmul(fc, weights['out']), biases['out'])
    return output

## 参数定义

In [6]:
# 每次是用100条数据进行训练
batch_size = 100

X = tf.placeholder(tf.float32, [None, 28*28])
Y = tf.placeholder(tf.float32)

## 使用数据集训练卷积神经网络

In [7]:
def train_neural_network(X, Y):
    predict = convolutional_neural_network(X)
    cost_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=Y))
    optimizer = tf.train.AdamOptimizer().minimize(cost_func) # learning rate 默认 0.001
    
    epochs = 1
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        epoch_loss = 0
        for epoch in range(epochs):
            for i in range(int(mnist.train.num_examples/batch_size)):
                x, y = mnist.train.next_batch(batch_size)
                _, c = session.run([optimizer, cost_func], feed_dict={X:x,Y:y})
                epoch_loss += c
            print(epoch, ' : ', epoch_loss)
            
        correct = tf.equal(tf.argmax(predict, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        print('准确率 : ', accuracy.eval({X:mnist.test.images, Y:mnist.test.labels}))

In [8]:
train_neural_network(X, Y)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0  :  1261164.5277404785
准确率 :  0.9273
